## Converting Xarray to CuPy


In [1]:
import cupy as cp
import xarray as xr
import numpy as np

In [2]:
cp.__version__, np.__version__, xr.__version__

('10.5.0', '1.22.4', '2022.3.0')

### 1. Passing CuPy array to Xarray's DataArrays

In [4]:
x = cp.arange(6).reshape(2, 3).astype('f')
# y = cp.ones((2, 3), dtype='float32')
z = np.ones((2, 3), dtype='int32')

In [5]:
lat = [40, 50]
lon = [100, 120, 140]
ds = xr.Dataset()

ds['x'] = xr.DataArray(x, dims=['lat', 'lon'], coords={'lat': lat, 'lon': lon})
ds['z'] = xr.DataArray(z, dims=['lat', 'lon'], coords={'lat': lat, 'lon': lon})
ds

<xarray.Dataset>
Dimensions:  (lat: 2, lon: 3)
Coordinates:
  * lat      (lat) int64 40 50
  * lon      (lon) int64 100 120 140
Data variables:
    x        (lat, lon) float32 array([[0., 1., 2.],        [3., 4., 5.]], dt...
    z        (lat, lon) int32 1 1 1 1 1 1

In [6]:
ds.x.data.device

<CUDA Device 0>

### 2. Converting each Coordinate

In [11]:
# Read a netCDF file
original_ds = xr.open_dataset("./../geocat-comp/test/eofunc_dataset.nc")
original_ds
print(type(original_ds.time.data))
# Create a placeholder xarray.Dataset
ds = xr.zeros_like(original_ds)
ds

<class 'numpy.ndarray'>


<xarray.Dataset>
Dimensions:  (time: 127, lat: 91, lon: 180, ncl3: 5, ncl4: 91, ncl5: 180,
              ncl6: 5, ncl7: 127)
Coordinates:
  * time     (time) int32 0 1 2 3 4 5 6 7 8 ... 119 120 121 122 123 124 125 126
  * lat      (lat) float32 -90.0 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 90.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
Dimensions without coordinates: ncl3, ncl4, ncl5, ncl6, ncl7
Data variables:
    sst      (time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    evec     (ncl3, ncl4, ncl5) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    tsout    (ncl6, ncl7) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [9]:
# Move the read netCDF data to the GPU
gpu_time_array = cp.asarray(original_ds.time.data)
print(type(gpu_time_array))
# Assign the GPU array to the TEMP variable in our new xarray.Dataset
ds['time'] = xr.DataArray(gpu_time_array)

<class 'cupy._core.core.ndarray'>


In [10]:
# Confirm that we have our TEMP variable is pointing to data on the GPU
ds.time.data.device

<CUDA Device 0>

### 3. Converting Variables 
This one can be done with two approaches.
#### a. Using cupy_xarray
github repo: https://github.com/xarray-contrib/cupy-xarray

In [15]:
import xarray as xr
import cupy_xarray  # This registers the `DataSet.cupy` and `DataArray.cupy` namespaces but is not used directly

ds = xr.open_dataset("./../geocat-comp/test/eofunc_dataset.nc")
print(type(ds.sst.data))  # numpy.ndarray

#ds = ds.cupy.as_cupy() #or 
ds = ds.as_cupy() 
print(type(ds.sst.data))  # cupy.core.core.ndarray

<class 'numpy.ndarray'>
<class 'cupy._core.core.ndarray'>



#### b. Using CuPy's function and convert each variable seperately
cp.asarray(variable)

In [20]:
import xarray as xr
import cupy as cp

ds = xr.open_dataset("./../geocat-comp/test/eofunc_dataset.nc")
print(type(ds.sst.data))  # numpy.ndarray

ds.sst.data  = cp.asarray(ds.sst.data)
ds.evec.data  = cp.asarray(ds.evec.data)
ds.tsout.data  = cp.asarray(ds.tsout.data)
print(type(ds.sst.data))  # cupy.core.core.ndarray
print(type(ds.evec.data))
print(type(ds.tsout.data))

<class 'numpy.ndarray'>
<class 'cupy._core.core.ndarray'>
<class 'cupy._core.core.ndarray'>
<class 'cupy._core.core.ndarray'>
